# Covered Call Optimizer

Below is the code that loads, runs, and analyzes the covered call optimization shown in the reddit post (link: https://www.reddit.com/r/options/comments/1ktwu86/i_built_a_script_to_get_the_best_covered_call/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button)

Ensure you have run pip install -r requirements.txt and that you have an active schwab api account.

#### Part I
Ensure that you are running activate_tokens.py in the background.

Load in libraries, symbols for the stocks you want to analyze, and your refresh tokens. 

In [ ]:
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import json
from datetime import datetime
import math
from scipy.stats import norm
load_dotenv()


AUTHORIZATION_TOKEN = os.getenv("AUTHORIZATION_TOKEN")
APP_KEY = os.getenv("APP_KEY")
APP_SECRET = os.getenv("APP_SECRET")

with open("stocks.txt", "r") as r:
    symbols = [line.strip() for line in r.readlines()]

with open("refresh_tokens.txt", "r") as r:
    data = json.load(r)


#### Part II

Run the below cell to bring the underlying price for the symbols defined in stocks.txt

In [4]:
url = "https://api.schwabapi.com/marketdata/v1/quotes"
token = data['tokens']['access_token']
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}"
}

all_quotes = {}
batch_size = 100

for i in range(0, len(symbols), batch_size):
    params = {
        "symbols": ", ".join(symbols[i:i+batch_size]),
        "fields": "quote,reference",
        "indicative": "false"
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        all_quotes.update(response.json())
    else:
        print(f"Error fetching data for batch {i//batch_size + 1}: {response.status_code}")

#### Part III

Choose the amount of strikes you want to bring for each stock and choose the month for which you want to analyze the contracts. Wherever there is a comment that says  # User Choice ensure you have selected your desired input

In [ ]:
option_chain_url = "https://api.schwabapi.com/marketdata/v1/chains"
with open("refresh_tokens.txt", "r") as r:
    data = json.load(r)
token = data['tokens']['access_token']
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}",
}


strikeCount = 30 # User Choice: choose how many strikes you want to bring for each stock
final_df = []
for symbol in list(all_quotes.keys())[:-2]:
    print(f"getting option chain with {strikeCount} strikes for {symbol}")
    params = {
    "symbol": symbol,
    "contractType": "CALL",
    "includeQuotes": "TRUE",
    'strikeCount':strikeCount,
    'expMonth':"MAY",} # User Choice: put the month you want the expiration to be (e.g. MAY, JUN, etc.)

    response = requests.get(option_chain_url, headers=headers, params=params)
    try:
        if response.status_code == 200:
            data = response.json()
            exps = list(data['callExpDateMap'].keys())
            print(f"     expirations: {exps}")
            the_exp = exps[1] # User Choice: choose the exp within the month you want to analyze
            days_to_expiration = int(the_exp.split(":")[1])
            strikes = pd.DataFrame.from_dict(data['callExpDateMap'][the_exp]).T.index.astype(float)
            df = pd.DataFrame.from_dict(data['callExpDateMap'][the_exp]).T 
            df = pd.json_normalize(df.to_dict(orient='records'))
            df.columns = [x.split(".")[1] for x in df.columns]
            df['daysToExpiration'] = days_to_expiration
            df['underlyingPrice'] = float(data['underlyingPrice'])
            df['strike']=strikes
            df['stockReturnCap'] =  (df['strike'] / df['underlyingPrice'] - 1).clip(lower=0)
            df['premiumReturn'] = np.where(df['strike'] < df['underlyingPrice'], 
                                        (df['bid'] / df['underlyingPrice']) - (df['underlyingPrice'] / df['strike'] - 1), 
                                        (df['bid'] / df['underlyingPrice'])) 

            df['annualizedPremiumReturn'] = (1 + df['premiumReturn']) ** (252/days_to_expiration) - 1
            df['costForUnderlying'] = df['underlyingPrice'] * 100
            df['breakevenPoint'] = df['underlyingPrice'] - df['bid']
            df['downsideProtection'] = df['bid'] / df['underlyingPrice']
            df['dollarReturn'] = df['premiumReturn'] * df['costForUnderlying']
            final_df.append(df)
        else:
            print("Error:", response.status_code, response.text)
    except Exception as e:
        print(e)
        
final_df = pd.concat(final_df, axis = 0)

Below is a way to calculate the risk neutral probability of profit. It utilizes and assumes current IV and calculates the cumulative distribution function up to the breakeven point for each row

In [ ]:
def compute_probs(row):
    S = row['underlyingPrice']
    K = row['breakevenPoint']
    vol = row['volatility'] / 100
    T = row['daysToExpiration'] / 365.0
    sigma_T = vol * math.sqrt(T)
    z = (math.log(K / S)) / sigma_T
    return 1 - norm.cdf(z)
final_df['probFullProfit'] = final_df.apply(compute_probs, axis= 1)
final_df

Below is an example of how to filter and sort by the risk measures you care about

In [ ]:
summary_df = final_df[(final_df.annualizedPremiumReturn > 0.5) & (final_df.downsideProtection > 0.05)]
summary_df[['description','bid','last','strike','underlyingPrice',
         'premiumReturn','annualizedPremiumReturn','costForUnderlying',
         'breakevenPoint','downsideProtection','dollarReturn','probFullProfit']].sort_values(by='downsideProtection')